In [16]:
#import packages 
import pandas as pd
import os
import csv
import re
from translate import Translator
import spacy
import langid
import keras_bert
import tensorflow as tf
import numpy as np
import time

#dataset
cwd = os.getcwd()
daen = pd.read_csv('New_balance_valid_url.csv')

In [17]:
# change the column orders
mydata = daen[['screen_name','country_code','country_name','tweet','favourites_count','followers_count','statuses_count','friends_count','listed_count','bounding_box','image_url','banner_url','centroid']]
mydata.head()

,screen_name,country_code,country_name,tweet,favourites_count,followers_count,statuses_count,friends_count,listed_count,bounding_box,image_url,banner_url,centroid
0,USGS_TexasRain,US,United States,"#USGS07343000 - N Sulphur Rv nr Cooper, TX \nH...",0,1377,34433,1,69,"[[[-106.645646, 25.837092], [-106.645646, 36.5...",https://pbs.twimg.com/profile_images/875796316...,https://pbs.twimg.com/profile_banners/71521146...,"(31.1688935, -100.0768885)"
1,GetYoPrimeOn,US,United States,https://t.co/ONEYjodpYc,2771,808,27208,680,3,"[[[-80.196625, 26.150489], [-80.196625, 26.209...",https://pbs.twimg.com/profile_images/112552666...,https://pbs.twimg.com/profile_banners/32416638...,"(26.1799205, -80.156826)"
2,marcsuckstoes,US,United States,this still Denzels best performance,225,117,1389,203,0,"[[[-76.22808, 36.672684], [-76.22808, 36.93239...",https://pbs.twimg.com/profile_images/130316925...,https://pbs.twimg.com/profile_banners/53844606...,"(36.8025415, -76.069946)"
3,seanmarnold,US,United States,@annkozma723 @sdtitmas @IanStuart66 @mtholfsen...,6518,3192,6096,1957,79,"[[[-74.041878, 40.570842], [-74.041878, 40.739...",https://pbs.twimg.com/profile_images/121654772...,https://pbs.twimg.com/profile_banners/18243997...,"(40.655138, -73.9487755)"
4,DRmetwatch,US,United States,@MatthewCappucci Don’t try. It’s not worth th...,92284,1962,14132,889,119,"[[[-77.220557, 39.053158], [-77.220557, 39.120...",https://pbs.twimg.com/profile_images/117405337...,https://pbs.twimg.com/profile_banners/32468285...,"(39.08705500000001, -77.16310250000001)"


In [18]:
#collect the tweet variable
no_url_data= pd.DataFrame(mydata.tweet) 
print(no_url_data)

                                                   tweet
0      #USGS07343000 - N Sulphur Rv nr Cooper, TX \nH...
1                                https://t.co/ONEYjodpYc
2                    this still Denzels best performance
3      @annkozma723 @sdtitmas @IanStuart66 @mtholfsen...
4      @MatthewCappucci Don’t try.  It’s not worth th...
5         Damn bro this month beat my ass. I’m exhausted
6      @MinhyVa @TunnelVa Hi! I don’t think we’ve tru...
7      Amazing Grace is a Christian hymn\nHighly inap...
8          @MadMunchkinArt I'm very sorry for your loss.
9      Somebody tell my momma I made it 😭🥳 https://t....
10                               https://t.co/Ig0Nujef5G
11     A small misunderstanding can show you exactly ...
12     Mitt Romney is doing his job. Sorry you don’t ...
13            808 Detroit Street https://t.co/Q4ntMUkQtl
14                                                 SRT 🔥
15     Looking to score a job at Harbor Freight Tools...
16                             

In [19]:
#delete url in tweet
no_url=no_url_data['tweet'].str.replace('http\S+|www.\S+', '', case=False)
print(no_url)

0        #USGS07343000 - N Sulphur Rv nr Cooper, TX \nH...
1                                                         
2                      this still Denzels best performance
3        @annkozma723 @sdtitmas @IanStuart66 @mtholfsen...
4        @MatthewCappucci Don’t try.  It’s not worth th...
5           Damn bro this month beat my ass. I’m exhausted
6        @MinhyVa @TunnelVa Hi! I don’t think we’ve tru...
7        Amazing Grace is a Christian hymn\nHighly inap...
8            @MadMunchkinArt I'm very sorry for your loss.
9                     Somebody tell my momma I made it 😭🥳 
10                                                        
11       A small misunderstanding can show you exactly ...
12       Mitt Romney is doing his job. Sorry you don’t ...
13                                     808 Detroit Street 
14                                                   SRT 🔥
15       Looking to score a job at Harbor Freight Tools...
16                                                      

In [20]:
#remove punctuation characters except hashtag
no_punc = no_url.str.replace('[^\w\s#]', '', flags=re.UNICODE)
print(no_punc)

0        #USGS07343000  N Sulphur Rv nr Cooper TX \nHea...
1                                                         
2                      this still Denzels best performance
3        annkozma723 sdtitmas IanStuart66 mtholfsen Ada...
4        MatthewCappucci Dont try  Its not worth the ef...
5             Damn bro this month beat my ass Im exhausted
6        MinhyVa TunnelVa Hi I dont think weve truly in...
7        Amazing Grace is a Christian hymn\nHighly inap...
8               MadMunchkinArt Im very sorry for your loss
9                       Somebody tell my momma I made it  
10                                                        
11       A small misunderstanding can show you exactly ...
12       Mitt Romney is doing his job Sorry you dont li...
13                                     808 Detroit Street 
14                                                    SRT 
15       Looking to score a job at Harbor Freight Tools...
16                                                      

In [21]:
#add the new tweets column
mydata['tweets']=no_punc
mydata.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,screen_name,country_code,country_name,tweet,favourites_count,followers_count,statuses_count,friends_count,listed_count,bounding_box,image_url,banner_url,centroid,tweets
0,USGS_TexasRain,US,United States,"#USGS07343000 - N Sulphur Rv nr Cooper, TX \nH...",0,1377,34433,1,69,"[[[-106.645646, 25.837092], [-106.645646, 36.5...",https://pbs.twimg.com/profile_images/875796316...,https://pbs.twimg.com/profile_banners/71521146...,"(31.1688935, -100.0768885)",#USGS07343000 N Sulphur Rv nr Cooper TX \nHea...
1,GetYoPrimeOn,US,United States,https://t.co/ONEYjodpYc,2771,808,27208,680,3,"[[[-80.196625, 26.150489], [-80.196625, 26.209...",https://pbs.twimg.com/profile_images/112552666...,https://pbs.twimg.com/profile_banners/32416638...,"(26.1799205, -80.156826)",
2,marcsuckstoes,US,United States,this still Denzels best performance,225,117,1389,203,0,"[[[-76.22808, 36.672684], [-76.22808, 36.93239...",https://pbs.twimg.com/profile_images/130316925...,https://pbs.twimg.com/profile_banners/53844606...,"(36.8025415, -76.069946)",this still Denzels best performance
3,seanmarnold,US,United States,@annkozma723 @sdtitmas @IanStuart66 @mtholfsen...,6518,3192,6096,1957,79,"[[[-74.041878, 40.570842], [-74.041878, 40.739...",https://pbs.twimg.com/profile_images/121654772...,https://pbs.twimg.com/profile_banners/18243997...,"(40.655138, -73.9487755)",annkozma723 sdtitmas IanStuart66 mtholfsen Ada...
4,DRmetwatch,US,United States,@MatthewCappucci Don’t try. It’s not worth th...,92284,1962,14132,889,119,"[[[-77.220557, 39.053158], [-77.220557, 39.120...",https://pbs.twimg.com/profile_images/117405337...,https://pbs.twimg.com/profile_banners/32468285...,"(39.08705500000001, -77.16310250000001)",MatthewCappucci Dont try Its not worth the ef...


In [22]:
#clean data
data = mydata[['screen_name','country_code','country_name','tweets','favourites_count','followers_count','statuses_count','friends_count','listed_count','bounding_box','image_url','banner_url','centroid']]
data

,screen_name,country_code,country_name,tweets,favourites_count,followers_count,statuses_count,friends_count,listed_count,bounding_box,image_url,banner_url,centroid
0,USGS_TexasRain,US,United States,#USGS07343000 N Sulphur Rv nr Cooper TX \nHea...,0,1377,34433,1,69,"[[[-106.645646, 25.837092], [-106.645646, 36.5...",https://pbs.twimg.com/profile_images/875796316...,https://pbs.twimg.com/profile_banners/71521146...,"(31.1688935, -100.0768885)"
1,GetYoPrimeOn,US,United States,,2771,808,27208,680,3,"[[[-80.196625, 26.150489], [-80.196625, 26.209...",https://pbs.twimg.com/profile_images/112552666...,https://pbs.twimg.com/profile_banners/32416638...,"(26.1799205, -80.156826)"
2,marcsuckstoes,US,United States,this still Denzels best performance,225,117,1389,203,0,"[[[-76.22808, 36.672684], [-76.22808, 36.93239...",https://pbs.twimg.com/profile_images/130316925...,https://pbs.twimg.com/profile_banners/53844606...,"(36.8025415, -76.069946)"
3,seanmarnold,US,United States,annkozma723 sdtitmas IanStuart66 mtholfsen Ada...,6518,3192,6096,1957,79,"[[[-74.041878, 40.570842], [-74.041878, 40.739...",https://pbs.twimg.com/profile_images/121654772...,https://pbs.twimg.com/profile_banners/18243997...,"(40.655138, -73.9487755)"
4,DRmetwatch,US,United States,MatthewCappucci Dont try Its not worth the ef...,92284,1962,14132,889,119,"[[[-77.220557, 39.053158], [-77.220557, 39.120...",https://pbs.twimg.com/profile_images/117405337...,https://pbs.twimg.com/profile_banners/32468285...,"(39.08705500000001, -77.16310250000001)"
5,_iaritaa_,US,United States,Damn bro this month beat my ass Im exhausted,62164,873,21046,543,7,"[[[-80.442683, 25.992486], [-80.442683, 26.063...",https://pbs.twimg.com/profile_images/129758193...,https://pbs.twimg.com/profile_banners/15940407...,"(26.0278895, -80.3279985)"
6,ConlinRei,US,United States,MinhyVa TunnelVa Hi I dont think weve truly in...,13904,666,1913,648,3,"[[[-112.014232, 33.508746], [-112.014232, 33.5...",https://pbs.twimg.com/profile_images/104253400...,https://pbs.twimg.com/profile_banners/71847737...,"(33.5456875, -111.9655245)"
7,Bgivey,US,United States,Amazing Grace is a Christian hymn\nHighly inap...,7895,3251,17260,4841,9,"[[[-124.059741, 46.433773], [-124.059741, 46.5...",https://pbs.twimg.com/profile_images/116006262...,https://pbs.twimg.com/profile_banners/32354348...,"(46.497139000000004, -124.0443065)"
8,FoxMaysville,US,United States,MadMunchkinArt Im very sorry for your loss,25504,95,25592,150,0,"[[[-89.57151, 36.497129], [-89.57151, 39.14735...",https://pbs.twimg.com/profile_images/118065621...,https://pbs.twimg.com/profile_banners/24414097...,"(37.822244, -85.7682405)"
9,fiendclubla,US,United States,Somebody tell my momma I made it,770,30,1700,69,0,"[[[-118.668404, 33.704538], [-118.668404, 34.3...",https://pbs.twimg.com/profile_images/127800337...,https://pbs.twimg.com/profile_banners/86072145...,"(34.0207895, -118.4119065)"


In [23]:
#install the bert-tensorflow package
!pip install bert-tensorflow

In [24]:
#get tweets from dataframe
def get_tweets_list(df):
    all_tweets = []
    s_name = []
    for num in range(len(df)):
        all_tweets.append(df.iloc[num]['tweets'])
        s_name.append(df.iloc[num]['screen_name'])
        print('tweets extracted')
    return all_tweets,s_name

#input list of tweets with structure as [[tweet, screen_name],[tweet, screen_name],[tweet, screen_name],...]
def get_bert_embeddings(tweet_list):
    model_path = "multi_cased_L-12_H-768_A-12"
    embeddings = keras_bert.extract_embeddings(model_path, tweet_list)
    print('embeddings complete')
    return(embeddings)

#mean pool the the embeddings to return 768 embeddings per sentence
def avg_pooling(embed_array):
    embeddings_pooled = []
    for sentence in embed_array:
        sentence = np.expand_dims((sentence),axis = 0)
        sentence = tf.keras.layers.GlobalAveragePooling1D()(sentence)
        embeddings_pooled.append(np.squeeze(sentence))
    return(embeddings_pooled)

In [25]:
#extracted first 10 tweets and check the time.
start= time.time()
list_of_tweets,screen_name = get_tweets_list(data[:10])
bert_embeddings = get_bert_embeddings(list_of_tweets)
bert_pooled = avg_pooling(bert_embeddings)
print(time.time() - start)

tweets extracted
tweets extracted
tweets extracted
tweets extracted
tweets extracted
tweets extracted
tweets extracted
tweets extracted
tweets extracted
tweets extracted
embeddings complete
23.88650894165039


In [26]:
#bert embedding
bert_embeddings_df = pd.DataFrame(list(zip(screen_name[0:12649],bert_pooled)),columns=['screen_name','Bert_embeddings'])
bert_embeddings_df.head()

,screen_name,Bert_embeddings
0,USGS_TexasRain,"[0.061395407, -0.66134953, 0.23918697, 0.14394..."
1,GetYoPrimeOn,"[0.12018438, -0.15244184, 1.0111687, 0.3777812..."
2,marcsuckstoes,"[-0.30565566, -0.29011664, -0.3280666, 0.38237..."
3,seanmarnold,"[-0.35415575, -0.6776336, 0.48186308, 0.352772..."
4,DRmetwatch,"[-0.31052125, 0.06173013, 0.7455791, 0.2770891..."
